In [ ]:
cd ../src

In [ ]:
import math
import numpy as np
import os
import pandas as pd

from matplotlib import pyplot as plt

from config import \
    TEMPERATURES, COOLING_RATES, \
    CONST_COOLING_RATE, CONST_TEMPERATURE


In [ ]:

plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [ ]:
results = []
for f in os.listdir("../results"):
    if f.endswith(".csv") and f.startswith("rand"):
        df = pd.read_csv(f"../results/{f}", header=0)
        results.append(df)
df = pd.concat(results)
cr_df = df.loc[df["temperature"] == CONST_TEMPERATURE]
t_df = df.loc[df["cooling_rate"] == CONST_COOLING_RATE]
t_df.head()

In [ ]:
def plot_result(
    sa_metric_name: str,
    cities_metric_name: str,
    dependent_var: str,
    relative_scale: bool = False,
    relative_to_base: bool = False,
    yscale: str = "linear"
) -> None:
    data_frame = t_df if sa_metric_name == "temperature" else cr_df
    sa_metrics = TEMPERATURES if sa_metric_name == "temperature" else COOLING_RATES
    cities_metrics = sorted(data_frame[cities_metric_name].unique())
    
    # Loop through each of the temperatures or cooling rates
    results = []
    for sa_metric in sa_metrics:
        curr_data_frame = data_frame.loc[data_frame[sa_metric_name] == sa_metric]

        # Loop through the city count or average city distance difference
        result = []
        for cities_metric in cities_metrics:
            vals = curr_data_frame.loc[curr_data_frame[cities_metric_name] == cities_metric]
            result.append(vals[dependent_var].mean())
        
        results.append(result)

    # If we want to rescale the y axis to be relative to the average
    if relative_scale or relative_to_base:
        n = len(sa_metrics)
        m = len(cities_metrics)
        weighted_vals = []
        for i in range(m):
            # Find the values
            vals = [results[j][i] for j in range(n)]
            avg_val = sum(vals) / len(vals)
            for j in range(n):
                results[j][i] /= avg_val
            
            weighted_vals.append(np.average([(1 / results[j][i]) * j for j in range(n)]))

        print(weighted_vals)
    if relative_to_base:
        results = np.array(results).T
        for i, result in enumerate(results):
            results[i] = 1 / (result *  (1 / result[0]))
        results = results.T
        for result in results:
            print([round(i, 3) for i in result])

    # Plot everything
    label_name = " ".join([s.capitalize() for s in sa_metric_name.split("_")])
    for i, result in enumerate(results):
        print(f"{sa_metrics[i]} avg relative: {np.average(result)}")
        plt.plot(cities_metrics, result, label=f"{label_name}: {sa_metrics[i]}")
    


    # Add labels, scaling, and save image
    plt.xlabel(" ".join([s.capitalize() for s in cities_metric_name.split("_")]))
    ylabel = dependent_var.capitalize()
    if relative_scale:
        ylabel = f"Relative {ylabel}"
    elif relative_to_base:
        ylabel = "Relative Optimality"
    plt.ylabel(ylabel)
    plt.xscale("log")
    plt.yscale(yscale)
    plt.legend()

    # Save image
    img_name = f"rand_{sa_metric_name}_{cities_metric_name}_{dependent_var}"
    if relative_scale:
        img_name += "_relative"
    elif relative_to_base:
        img_name += "_relative-to-base"
    plt.savefig(f"../report/images/{img_name}.jpg", dpi=144)
    
    plt.show()


In [ ]:
plot_result("cooling_rate", "city_count", "distance", relative_to_base=True, yscale="linear")

In [ ]:
plot_result("cooling_rate", "city_count", "iterations", relative_scale=True, yscale="log")

In [ ]:
plot_result("temperature", "avg_dist_diff", "distance", relative_scale=False)

In [ ]:
plot_result("temperature", "avg_dist_diff", "iterations", relative_scale=False)